In [256]:
import numpy as np
from tqdm.auto import tqdm
from time import time
import json, pickle, os, string, kenlm, json
from collections import defaultdict, Counter
from itertools import groupby
import Levenshtein as Lev
import math


In [237]:
def lse(*args):
  """
  Stable log sum exp.
  """
  if all(a == -float('inf') for a in args):
      return -float('inf')
  a_max = max(args)
  lsp = math.log(sum(math.exp(a - a_max)
                      for a in args))
  return a_max + lsp

In [238]:
#s1 = True text
#s2 = predicted text

def wer_(s1, s2):
    """
    Computes the Word Error Rate, defined as the edit distance between the
    two provided sentences after tokenizing to words.
    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """

    # build mapping of words to integers
    b = set(s1.split() + s2.split())
    word2char = dict(zip(b, range(len(b))))

    # map the words to a char array (Levenshtein packages only accepts
    # strings)
    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]
    
    return Lev.distance(''.join(w1), ''.join(w2))

def cer_(s1, s2):
    """
    Computes the Character Error Rate, defined as the edit distance.

    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """
    s1, s2, = s1.replace(' ', ''), s2.replace(' ', '')

    return Lev.distance(s1, s2)




#When using the above implementation, use the code belove to calculate the wer in percentatge: 
#pred = list of ouput prediction of model (it is the text) # example [" MY NAME IS HEMANT", " I AM A GOD"]
# total_wer = 0
# for x in range(len(pred)):
#     transcript, reference = data_[x][1], pred[x]
#     wer_inst = wer(transcript, reference)
#     total_wer += float(wer_inst)
# print("WER is : ",total_wer/len(pred),"%")
# wer_(pred,true)/len(pred.split(' '))*100

In [239]:
out = np.load("out.npy")
with open("true.txt", "r") as f:
    reference = f.read()
    
with open("pred.txt", "r") as f:
    transcript = f.read()

In [240]:
def ctc_best_path(out,labels):
    "implements best path decoding as shown by Graves"
    out = [labels[i] for i in np.argmax(out, axis=1) if i!=labels[-1]]
    o = ""
    for i,j in groupby(out):
        o = o + i
    return o.replace("_","")

In [241]:
labels = "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ "

In [242]:
gred_txt = ctc_best_path(out,labels)
print(gred_txt)
wer_(gred_txt,reference)/len(reference.split(' '))*100

OUR PRODATIVITY AND OUR COMMITMENT TO OUR CLIENTS XXXUM TO MAKE SURE THAT WE NEED WHAT BE XXXUM PROMISED FOR THE DEADLINE


19.047619047619047

### WORD LM Implementation

In [315]:
def ctc_beam_search(out,labels, prune=0.0001, k=20, lm=None,alpha=0.3,beta=12):
    
    bc_i = 0 # blank/special charatcter index 
    F = out.shape[1]
    out = np.log(np.vstack((np.zeros(F), out)))
    steps = out.shape[0]
    
    pb, pnb = defaultdict(Counter), defaultdict(Counter)
    pb[0][''], pnb[0][''] = 0, -float("inf")
    prev_beams = ['']
    for t in tqdm(range(1,steps)):
        for b in prev_beams:           
            for c_t in labels:
                index = labels.index(c_t)
                if c_t == "_": #Extending with a blank
                    pb[t][b] = lse(pb[t][b],out[t][index]+pb[t-1][b], out[t][index]+pnb[t-1][b])
                    continue
                
                i_plus = b + c_t
#                 print(t,'--',i_plus,'\n')
                if len(b.replace(' ', '')) > 0  and c_t == b[-1]: #Extending with the same character as the last one
                    pnb[t][b] = lse(out[t][index]+pnb[t-1][b],pnb[t][b])
                    pnb[t][i_plus] = lse(pnb[t][i_plus],out[t][index]+pb[t-1][b])
                elif c_t == " " and len(b.replace(' ', '')) > 0 : # LM constraints
                    print(t,'--',c_t,'----',i_plus,'\n')
#                     prob = [i[0] for i in lm.full_scores(i_plus,eos=False,bos=False)][-1]
#                     lm_p = (10**prob)**alpha
#                     pnb[t][i_plus] += lm_p*out[t][index]*(pb[t-1][b] + pnb[t-1][b])*(len(b.split())+1)**beta
                    pnb[t][i_plus] = lse(pnb[t][i_plus],out[t][index]+pb[t-1][b],out[t][index] + pnb[t-1][b])
                else:
                    pnb[t][i_plus] = lse(pnb[t][i_plus],out[t][index]+pb[t-1][b], out[t][index]+pnb[t-1][b])
                    
                # If the new beam is not in the previous beams
#                 if i_plus not in prev_beams:
#                     pb[t][i_plus] = lse(pb[t][i_plus],out[t][labels.index("_")]+pb[t - 1][i_plus], out[t][labels.index("_")]+ pnb[t - 1][i_plus])
#                     pnb[t][i_plus] = lse(pnb[t][i_plus],out[t][index] + pnb[t - 1][i_plus])

        ptot = pb[t] + pnb[t]
        print(t,'---',ptot,'\n')
        prev_beams = sort_beam(ptot,k)
        print(t,'---',prev_beams,'\n')
    return prev_beams[0], pb, pnb


In [316]:
beam_txt, pb, pnb = ctc_beam_search(out[:15],labels,0,k=1,lm=lm_w,alpha=0,beta=0)

/home/hemant/.conda/envs/deep/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


1 --- Counter({'': 0.6929092105053745, 'S': 0.00024981729039509143, 'T': 7.424721963053679e-05, 'I': 5.9175736937318984e-05, 'N': 1.8866742094371015e-05, 'M': 1.849540780159389e-05, 'W': 1.561804483666136e-05, 'A': 1.5216571870643262e-05, 'B': 4.4253817882818795e-06, 'Y': 3.741074402393878e-06, 'F': 3.657847077505103e-06, 'X': 3.23419317241585e-06, 'P': 1.991103381893167e-06, 'C': 1.2294217813686257e-06, 'R': 1.0764848529397004e-06, 'G': 9.655448024889328e-07, 'U': 7.547512262425191e-07, 'H': 7.164367437511687e-07, 'O': 6.900419755123499e-07, ' ': 6.83660737693999e-07, 'D': 4.988801157887766e-07, 'L': 3.214914608443128e-07, 'E': 1.7737963123885353e-07, 'V': 1.2090391245583605e-07, 'J': 9.95456680832672e-08, 'Q': 7.703289797992681e-08, 'K': 6.441670100801452e-08, 'Z': 1.5749090796263718e-08, "'": 7.094866891022199e-09}) 

1 --- [''] 

2 --- Counter({'': 1.38614668426133, 'S': 4.779856656685156e-05, 'H': 1.1940558746645468e-05, 'R': 9.323963212242155e-06, 'N': 8.5835310265424e-06, 'T': 7

In [299]:
print(beam_txt)
wer_(beam_txt,reference)/len(reference.split(' '))*100

OUR PRODATIVITY AND OUR COMMITMENT TO OUR CLIENTS XXXUM TO MAKE SURE THAT WE NEED WHAT BE XXXUM PROMISED FOR THE DEADLINE


19.047619047619047

In [115]:
lm_w = kenlm.LanguageModel('/home/hemant/ctc_decoders/3_gram_full.arpa')

In [140]:
def sort_beam(ptot,k):
    if len(ptot) < k:
        return [i for i in ptot.keys()]
    else:
        dict_ = sorted(dict((v,k) for k,v in ptot.items()).items(),reverse=True)[:k]
        return [i[1] for i in dict_]

#using WORD LM
def ctc_beam_search(out,labels, prune=0.0001, k=20, lm=None,alpha=0.3,beta=12):
    "implements CTC Prefix Search Decoding Algo13.043478260869565%'rithm as shown by Graves"
    '''
    out = ctc output
    labels = string of labels
    prune = prune the ctc output
    k=beam-width
    lm=word age model used
    alpha,beta = hyper-parameters
    '''

    bc_i = 0 # blank/special charatcter index 
    F = out.shape[1]
    out = np.log(np.vstack((np.zeros(F), out)))
    steps = out.shape[0]
    
    # time steps: ([beam, score]), for each time step we have a number of beams and each beam has a score.
    pb, pnb = defaultdict(Counter), defaultdict(Counter) 

#     pb[0][''], pnb[0][''] = 1, 0
    pb[0][''], pnb[0][''] = 0, -float("inf")
    prev_beams = ['']
#     print(out)
    for t in range(1,steps):
        pruned_alphabet = [labels[i] for i in np.where(out[t] > prune)[0]]
        for b in prev_beams:
            
            for c_t in labels: #pruned_alphabet:
                index = labels.index(c_t)
                #Collapsing case (copy case as the last character in the beam)
                if c_t == "_": #Extending with a blank
                    pb[t][b] = lse(pb[t][b],out[t][index]+pb[t-1][b], out[t][index]+pnb[t-1][b])
                    continue
                else:
                    i_plus = b + c_t
                    if len(b) > 0 and c_t == b[-1]: #Extending with the same character as the last one
                        pnb[t][b] = lse(out[t][index]+pnb[t-1][b],pnb[t][b])
                        pnb[t][i_plus] = lse(pnb[t][i_plus],out[t][index]*pb[t-1][b])
                    #Extending case as the last character is different
                    elif c_t == " " and len(b.replace(' ', '')) > 0 : # LM constraints
                        prob = [i[0] for i in lm.full_scores(i_plus,eos=False,bos=False)][-1]
                        lm_p = (10**prob)**alpha
                        pnb[t][i_plus] += lm_p*out[t][index]*(pb[t-1][b] + pnb[t-1][b])*(len(b.split())+1)**beta
                    else:
                        pnb[t][i_plus] = lse(pnb[t][i_plus],out[t][index]+pb[t-1][b], out[t][index]+pnb[t-1][b])
                    # If the new beam is not in the previous beams
                    if i_plus not in prev_beams:
                        pb[t][i_plus] = lse(pb[t][i_plus],out[t][labels.index("_")]+pb[t - 1][i_plus], out[t][labels.index("_")]+ pnb[t - 1][i_plus])
                        pnb[t][i_plus] = lse(pnb[t][i_plus],out[t][index] + pnb[t - 1][i_plus])

        ptot = pb[t] + pnb[t]
#         print(t,'---',pb,'\n',pnb,'\n',ptot,'\n')
        prev_beams = sort_beam(ptot,k)
    
    return prev_beams[0], pb, pnb

beam_txt, pb, pnb = ctc_beam_search(out[:15],labels,0,k=2,lm=lm_w,alpha=0,beta=0)


/home/hemant/.conda/envs/deep/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log


In [128]:
F = out.shape[1]
np.log(np.vstack((np.zeros(F), out)))[:15][1]

/home/hemant/.conda/envs/deep/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


array([-4.75996752e-04, -1.87638943e+01, -1.10931179e+01, -1.23281518e+01,
       -1.36089660e+01, -1.45108998e+01, -1.55449735e+01, -1.25186340e+01,
       -1.38505728e+01, -1.41489755e+01, -9.73496936e+00, -1.61226493e+01,
       -1.65578929e+01, -1.49502947e+01, -1.08979788e+01, -1.08781004e+01,
       -1.41865131e+01, -1.31268206e+01, -1.63790332e+01, -1.37418091e+01,
       -8.29465583e+00, -9.50807310e+00, -1.40968773e+01, -1.59282697e+01,
       -1.10670758e+01, -1.26417294e+01, -1.24961358e+01, -1.79664832e+01,
       -1.41958037e+01])

In [145]:
beam_txt, pb, pnb = ctc_beam_search(out,labels,0,k=100,lm=lm_w,alpha=0,beta=0)
print(beam_txt)
wer_(beam_txt,reference)/len(reference.split(' '))*100

/home/hemant/.conda/envs/deep/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log


 HOUR  PRODATIVVITYYAND  OOUR COMMMITMEEENTT TT OUR  CLIIEENTS XXUMMTOOMAAKEE  SURETHETT WEE  NEEDD WHAT BEXXXUMM PROMISSED  FOR TTEE DEADLINE


80.95238095238095

In [89]:
beam_txt

'SRERIHOXUWAREITSHSE PRORDACITIVIVITYE ANDOH OUVRT COMNMITENMEINTSYASUZNDSRTD TOU OUR CLINRNENENTSTANXOXO XUXUMUGDTOU MAKE SUREUTHTI WEYU NEAESLYLSYSYHR WATLILI EYBIATWXEX XAXUMHTM PROXOMBISED FOR TE DEADTFLINGNEHYOFDURITU'

In [67]:
np.log(out)

array([[-4.7599676e-04, -1.8763895e+01, -1.1093118e+01, ...,
        -1.2496136e+01, -1.7966484e+01, -1.4195804e+01],
       [-3.8266913e-05, -2.0051754e+01, -1.4749942e+01, ...,
        -1.6135670e+01, -2.2034241e+01, -1.8169502e+01],
       [-3.9935909e-05, -1.9766520e+01, -1.5708939e+01, ...,
        -1.6237494e+01, -2.2249647e+01, -1.9198616e+01],
       ...,
       [-3.5762793e-07, -1.9703728e+01, -1.9921610e+01, ...,
        -2.6936598e+01, -3.3852917e+01, -1.5329208e+01],
       [ 0.0000000e+00, -2.0324478e+01, -1.9751274e+01, ...,
        -2.8223896e+01, -3.6162498e+01, -1.6910114e+01],
       [-1.1920930e-07, -2.3349606e+01, -1.9762342e+01, ...,
        -2.5987900e+01, -3.7774834e+01, -1.6420919e+01]], dtype=float32)

In [ ]:
import os
os.chdir("/home/hemant/E2E_NER-Through-Speech/S2T/")
from opts import add_decoder_args, add_inference_args
from utils import load_model
import os
# from ctc_decoders import *
import argparse

import numpy as np
import torch
from tqdm import tqdm
from data.data_loader import SpectrogramParser

from opts import add_decoder_args, add_inference_args
from utils import load_model

In [ ]:
decoding = 'beam_w'
prune = 0.00001
beam_width = 50
alpha = 1
beta = 4
lm = kenlm.LanguageModel('/home/hemant/asr_wm/data/ner/align_wav_txt/4_gram.arpa')

torch.set_grad_enabled(False)
device = torch.device("cuda")
model = load_model(device, "/home/hemant/asr_wm/models/deep/final.pth")
spect_parser = SpectrogramParser(model.audio_conf, normalize=True)

torch.cuda.set_device(int(0))
with open("/home/hemant/asr_wm/dev.csv","r") as f:
    csv = f.readlines()

total_cer, total_wer, num_tokens, num_chars = 0, 0, 0, 0
lm = lm_w
output = []
from tqdm.auto import tqdm
for i in tqdm(csv[:1]):
    audio_path, reference_path = i.split(",")

    spect = spect_parser.parse_audio(audio_path).contiguous()
    spect = spect.view(1, 1, spect.size(0), spect.size(1))
    spect = spect.to(device)

    input_sizes = torch.IntTensor([spect.size(3)]).int()
    out, output_sizes = model(spect, input_sizes)
    out = out.cpu().detach().numpy()[0]

    transcript = ctc_beam_search(out,labels,prune,beam_width,lm,alpha,beta)
#     transcript = ctc_best_path(out,labels)
    with open(reference_path.replace("\n",""),"r") as f:
        reference = f.readline()
        
#     break
    output.append([transcript,reference])
    wer_inst = wer_(transcript,reference)
    cer_inst = cer_(transcript, reference)
    total_wer += wer_inst
    total_cer += cer_inst
    num_tokens += len(reference.split(' '))
    num_chars += len(reference.replace(' ', ''))

        
wer = (float(total_wer) / num_tokens)*100
cer = (float(total_cer) / num_chars)*100
print('Test Summary \t'
    'Average WER {wer:.3f}\t'
    'Average CER {cer:.3f}\t?'.format(wer=wer, cer=cer))

In [ ]:
alpha, beta = np.linspace(0.2,2,15), np.linspace(2,10,5)

In [ ]:
alpha

In [ ]:
beta

In [ ]:
alpha, beta = np.linspace(0.2,2,15), np.linspace(0,6,15)
values = [[i,j] for j in beta for i in alpha]

total_cer, total_wer, num_tokens, num_chars = 0, 0, 0, 0
lm = lm_w
output = []

for alpha,beta in tqdm(values):
    for i in csv:
        audio_path, reference_path = i.split(",")

        spect = spect_parser.parse_audio(audio_path).contiguous()
        spect = spect.view(1, 1, spect.size(0), spect.size(1))
        spect = spect.to(device)

        input_sizes = torch.IntTensor([spect.size(3)]).int()
        out, output_sizes = model(spect, input_sizes)
        out = out.cpu().detach().numpy()[0]

        if decoding == "greedy": transcript = ctc_best_path(out,labels)
        elif decoding == "beam_w": transcript = ctc_beam_search(out,labels,prune,beam_width,lm,alpha,beta)
        elif decoding == "beam_c": transcript = ctc_beam_search_clm(out,labels,prune,beam_width,lm,alpha=alpha,beta=beta)

        with open(reference_path.replace("\n",""),"r") as f:
            reference = f.readline()
        wer_inst = wer_(transcript,reference)
        cer_inst = cer_(transcript, reference)
        total_wer += wer_inst
        total_cer += cer_inst
        num_tokens += len(reference.split(' '))
        num_chars += len(reference.replace(' ', ''))
    np.save("/home/hemant/ctc_decoders/abw.npy",np.array(output))
    wer = (float(total_wer) / num_tokens)*100
    cer = (float(total_cer) / num_chars)*100
    output.append([alpha,beta,wer])
    print("aplha: ",alpha,"beta: ",beta)
    print('Test Summary \t'
        'Average WER {wer:.3f}\t'
        'Average CER {cer:.3f}\t'.format(wer=wer, cer=cer))